# __PotgreSQL y Docker con Python__

## Creación de la `conexión`y el `cursor`.


Cada una de las siguientes tablas y las respctivas consultas se realizaron desde Python usando la API `psycopg2`. Para esto, se creó una `conexión`y un `cursor` de manera previa:

In [1]:
import psycopg2
from psycopg2 import Error
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats

In [2]:
try:
    connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")

    cursor = connection.cursor()
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")

    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

PostgreSQL server information
{'user': 'Elianna_user', 'channel_binding': 'prefer', 'dbname': 'Elianna_db', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'} 

You are connected to -  ('PostgreSQL 16.2 (Debian 16.2-1.pgdg120+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) 

PostgreSQL connection is closed


Con el código anterior, primeramente se estableció la conexión con la API `psycopg2`, se proporcionan los párámetros para la conexión: `user`, `password`, `host`, `port` y `database` creadas previamente en el entorno virtul `Empresa` como instancia de base de datos en `Docker`. Se crea el `cursor` y la conexión con éste. Luego se ejecuta la consulta para seleccionar la versión del servicio `PotgreSQL`. 

## Creación de tablas: ``employees``.

I) Crear la tabla nombrada: ``employees``. _(codigo prestablecido en el ejercicio: Ver link en la introducción)._


In [3]:
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute('''DROP TABLE IF EXISTS employees''')
cursor.execute('''CREATE TABLE employees
   ( employee_id INTEGER
   , first_name VARCHAR(20)
   , last_name VARCHAR(25)
   , email VARCHAR(25)
   , phone_number VARCHAR(20)
   , hire_date DATE
   , job_id VARCHAR(10)
   , salary NUMERIC(8,2)
   , commission_pct NUMERIC(2,2)
   , manager_id INTEGER
   , department_id INTEGER
   ) ;
CREATE UNIQUE INDEX emp_emp_id_pk
         ON employees (employee_id) ;
ALTER TABLE employees ADD
   PRIMARY KEY (employee_id);
''')
connection.commit()
connection.close()

Una vez ejecutado el código anterior vamos a ``PgAdmin`` y verificamos en la ruta Servers > Empresa > Databases > Elianna_db > Schemas > public > Tables, que la tabla `employees` ha sido creada: 

<img src="C:\Users\Eliana Gomez\Pictures\Screenshots\cap1.png">
<p style="text-align:center;">Tabla employees en la base datos Elianna_db</p>

### Consulta del contenido de la tabla `employees`.

In [4]:

connection = psycopg2.connect(user="Elianna_user",
                                password="password",
                                host="localhost",
                                port="5432",
                                database="Elianna_db")
cursor = connection.cursor()

cursor.execute("SELECT * FROM employees LIMIT 10;")
r = cursor.fetchall()
print(r)
    
cursor.execute("SELECT count(1) FROM employees;")
r = cursor.fetchone()
print(r)

connection.close()


[]
(0,)


Con esta consulta se seleccionan ``todas las columnas`` de la tabla ``employees`` y devuelve las ``primeras 10 filas`` de resultados. Seguidamente, se cuenta el ``número total de filas`` en la tabla ``employees``. En este caso, dado que la tabla aún no contine información, se muestra un vector vacio y el conteo de filas en 0. 

## Creación de tablas: ``courses``. 


II) Crear la tabla ``courses`` con las siguientes columnas:

``course_id`` - integer y primary key.

``course_name`` - valores alfanuméricos o de cadena de hasta 60 caracteres.

``course_author`` - nombre del autor de hasta 40 caracteres.

``course_status`` - published, draft, inactive.

``course_published_dt ``- valor de tipo fecha.

In [5]:
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute('''DROP TABLE IF EXISTS courses''')
cursor.execute('''CREATE TABLE courses
    (course_id SERIAL PRIMARY KEY
   , course_name VARCHAR(60)
   , course_author VARCHAR(40)
   , course_status VARCHAR
   , course_published_dt DATE        
    );
''')

connection.commit()

connection.close()

Una vez ejecutado el código anterior vamos a ``PgAdmin`` y verificamos en la ruta Servers > Empresa > Databases > Elianna_db > Schemas > public > Tables, que la tabla `courses` ha sido creada: 

<img src="C:\Users\Eliana Gomez\Pictures\Screenshots\cap3.png">
<p style="text-align:center;">Tabla courses en la base datos Elianna_db</p>

### Inserción de datos.
III) Insertar datos

Inserte los datos en ``courses`` utilizando los datos proporcionados _(Ver link en la introducción)_. Asegúrese de que el ``id`` es generado por el sistema. No olvide refrescar la información de la base de datos.


In [6]:
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Programming using Python', 'Bob Dillon', 'published', '2020-09-30')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Data Engineering using Python', 'Bob Dillon', 'published', '2020-07-15')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Data Engineering using Scala', 'Elvis Presley', 'draft', NULL)");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Programming using Scala', 'Elvis Presley', 'published', '2020-05-12')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Programming using Java', 'Mike Jack', 'inactive', '2020-08-10')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Web Applications-Python Flask', 'Bob Dillon', 'inactive', '2020-07-20')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Web Applications-Java Scring', 'Mike Jack', 'draft',NULL)");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Pipeline Orchestration-Python', 'Bob Dillon', 'draft',NULL)");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Streaming Pipelines-Python', 'Bob Dillon', 'published', '2020-10-05')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Web Applications-Scala Play', 'Elvis Presley', 'inactive', '2020-09-30')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Web Applications-Python Django', 'Bob Dillon', 'published', '2020-06-23')");
cursor.execute("INSERT INTO courses (course_name, course_author, course_status, course_published_dt) VALUES ('Server Automation-Ansible', 'Uncle Sam', 'published', '2020-07-05')");

connection.commit()
connection.close()

Una vez ejecutado el código anterior vamos a ``PgAdmin`` y verificamos en la ruta Servers > Empresa > Databases > Elianna_db > Schemas > public > Tables > courses y seleccionamos la opción `All Rows` para ver la tabla y su contenido:

<img src="C:\Users\Eliana Gomez\Pictures\Screenshots\cap4.png">
<p style="text-align:center;">Tabla courses con valores insertados</p>

### Consultas: tabla `courses`.

IV) Borre todos los cursos que no estén en modo borrador ni publicados. Proporcione la sentencia de borrado como respuesta para este ejercicio en el ``Jupyter Book``. Para validar, obtenga el recuento de todos los cursos publicados por autor y asegúrese de que la salida está ordenada en forma descendente por recuento.

In [7]:
from tabulate import tabulate 
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
    SELECT COUNT(course_name),course_author 
    FROM courses 
    WHERE course_status <> 'inactive'
    GROUP BY course_author
    ORDER BY COUNT(course_name) DESC
    """);

r = cursor.fetchall()
print(tabulate(r, headers=['total published and draft', 'course_author'], tablefmt="simlple"))
connection.close()

  total published and draft  course_author
---------------------------  ---------------
                          5  Bob Dillon
                          2  Elvis Presley
                          1  Uncle Sam
                          1  Mike Jack


## Creación de tablas: `users`.




VI) Crear la base de datos ``users:`` 
 _(codigo proporcionado por el ejercicio. Ver link en la introducción)._

In [8]:
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()


cursor.execute('''DROP TABLE IF EXISTS users''')
cursor.execute('''CREATE TABLE users(
    user_id SERIAL PRIMARY KEY,
    user_first_name VARCHAR(30),
    user_last_name VARCHAR(30),
    user_email_id VARCHAR(50),
    user_gender VARCHAR(1),
    user_unique_id VARCHAR(15),
    user_phone_no VARCHAR(20),
    user_dob DATE,
    created_ts TIMESTAMP)
''');

connection.commit()
connection.close()

Una vez ejecutado el código anterior vamos a ``PgAdmin`` y verificamos en la ruta Servers > Empresa > Databases > Elianna_db > Schemas > public > Tables, que la tabla `users` ha sido creada: 

<img src="C:\Users\Eliana Gomez\Pictures\Screenshots\cap5.png">
<p style="text-align:center;">Tabla users en la base de datos Elianna_db</p>

### Inserción de datos.

Inserte los valores proporcionados: _(Ver link en la instrucción)._

In [9]:
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute('''
    insert into users (
    user_first_name, user_last_name, user_email_id, user_gender, 
    user_unique_id, user_phone_no, user_dob, created_ts)
    
    VALUES
    ('Giuseppe', 'Bode', 'gbode0@imgur.com', 'M', '88833-8759', 
     '+86 (764) 443-1967', '1973-05-31', '2018-04-15 12:13:38'),
    ('Lexy', 'Gisbey', 'lgisbey1@mail.ru', 'F', '262501-029', 
     '+86 (751) 160-3742', '2003-05-31', '2020-12-29 06:44:09'),
    ('Karel', 'Claringbold', 'kclaringbold2@yale.edu', 'F', '391-33-2823', 
     '+62 (445) 471-2682', '1985-11-28', '2018-11-19 00:04:08'),
    ('Marv', 'Tanswill', 'mtanswill3@dedecms.com', 'F', '1195413-80', 
     '+62 (497) 736-6802', '1998-05-24', '2018-11-19 16:29:43'),
    ('Gertie', 'Espinoza', 'gespinoza4@nationalgeographic.com', 'M', '471-24-6869', 
     '+249 (687) 506-2960', '1997-10-30', '2020-01-25 21:31:10'),
    ('Saleem', 'Danneil', 'sdanneil5@guardian.co.uk', 'F', '192374-933', 
     '+63 (810) 321-0331', '1992-03-08', '2020-11-07 19:01:14'),
    ('Rickert', 'O''Shiels', 'roshiels6@wikispaces.com', 'M', '749-27-47-52', 
     '+86 (184) 759-3933', '1972-11-01', '2018-03-20 10:53:24'),
    ('Cybil', 'Lissimore', 'clissimore7@pinterest.com', 'M', '461-75-4198', 
     '+54 (613) 939-6976', '1978-03-03', '2019-12-09 14:08:30'),
    ('Melita', 'Rimington', 'mrimington8@mozilla.org', 'F', '892-36-676-2', 
     '+48 (322) 829-8638', '1995-12-15', '2018-04-03 04:21:33'),
    ('Benetta', 'Nana', 'bnana9@google.com', 'M', '197-54-1646', 
     '+420 (934) 611-0020', '1971-12-07', '2018-10-17 21:02:51'),
    ('Gregorius', 'Gullane', 'ggullanea@prnewswire.com', 'F', '232-55-52-58', 
     '+62 (780) 859-1578', '1973-09-18', '2020-01-14 23:38:53'),
    ('Una', 'Glayzer', 'uglayzerb@pinterest.com', 'M', '898-84-336-6', 
     '+380 (840) 437-3981', '1983-05-26', '2019-09-17 03:24:21'),
    ('Jamie', 'Vosper', 'jvosperc@umich.edu', 'M', '247-95-68-44', 
     '+81 (205) 723-1942', '1972-03-18', '2020-07-23 16:39:33'),
    ('Calley', 'Tilson', 'ctilsond@issuu.com', 'F', '415-48-894-3', 
     '+229 (698) 777-4904', '1987-06-12', '2020-06-05 12:10:50'),
    ('Peadar', 'Gregorowicz', 'pgregorowicze@omniture.com', 'M', '403-39-5-869', 
     '+7 (267) 853-3262', '1996-09-21', '2018-05-29 23:51:31'),
    ('Jeanie', 'Webling', 'jweblingf@booking.com', 'F', '399-83-05-03', 
     '+351 (684) 413-0550', '1994-12-27', '2018-02-09 01:31:11'),
    ('Yankee', 'Jelf', 'yjelfg@wufoo.com', 'F', '607-99-0411', 
     '+1 (864) 112-7432', '1988-11-13', '2019-09-16 16:09:12'),
    ('Blair', 'Aumerle', 'baumerleh@toplist.cz', 'F', '430-01-578-5', 
     '+7 (393) 232-1860', '1979-11-09', '2018-10-28 19:25:35'),
    ('Pavlov', 'Steljes', 'psteljesi@macromedia.com', 'F', '571-09-6181', 
     '+598 (877) 881-3236', '1991-06-24', '2020-09-18 05:34:31'),
    ('Darn', 'Hadeke', 'dhadekej@last.fm', 'M', '478-32-02-87', 
     '+370 (347) 110-4270', '1984-09-04', '2018-02-10 12:56:00'),
    ('Wendell', 'Spanton', 'wspantonk@de.vu', 'F', null, 
     '+84 (301) 762-1316', '1973-07-24', '2018-01-30 01:20:11'),
    ('Carlo', 'Yearby', 'cyearbyl@comcast.net', 'F', null, 
     '+55 (288) 623-4067', '1974-11-11', '2018-06-24 03:18:40'),
    ('Sheila', 'Evitts', 'sevittsm@webmd.com', null, '830-40-5287',
     null, '1977-03-01', '2020-07-20 09:59:41'),
    ('Sianna', 'Lowdham', 'slowdhamn@stanford.edu', null, '778-0845', 
     null, '1985-12-23', '2018-06-29 02:42:49'),
    ('Phylys', 'Aslie', 'paslieo@qq.com', 'M', '368-44-4478', 
     '+86 (765) 152-8654', '1984-03-22', '2019-10-01 01:34:28')
''')
connection.commit()
connection.close()

Una vez ejecutado el código anterior vamos a ``PgAdmin`` y verificamos en la ruta Servers > Empresa > Databases > Elianna_db > Schemas > public > Tables > users y seleccionamos la opción `All Rows` para ver la tabla y su contenido:

<img src="C:\Users\Eliana Gomez\Pictures\Screenshots\cap6.png">
<p style="text-align:center;">Tabla users con valores insertados</p>

### Consultas: tabla `users`.



VII) Obtenga el número de usuarios creados por año. Utilice la tabla de usuarios para este ejercicio.

- La salida debe contener el año de 4 dígitos y el recuento.

- Use funciones específicas de fecha para obtener el año usando ``created_ts``.

- Asegúrese de definir alias a las columnas como ``created_year`` y ``user_count`` respectivamente.

- Los datos deben ordenarse de forma ascendente por ``created_year``.

- Cuando ejecutes la consulta usando el entorno ``Jupyter``, puede que tenga decimales para los enteros. Por lo tanto, puede mostrar los resultados incluso con decimales.



In [10]:
from tabulate import tabulate 
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
    SELECT COUNT (user_id) AS user_count, EXTRACT (YEAR FROM created_ts) AS created_year
    FROM users 
    GROUP BY created_year
    ORDER BY (created_year) ASC
    """);

r = cursor.fetchall()

print(tabulate(r, headers = ['user_id', 'created_year'], tablefmt="simple"))
connection.close()

  user_id    created_year
---------  --------------
       13            2018
        4            2019
        8            2020


VIII) Obtenga los días de nacimiento de todos los usuarios nacidos en el mes May.

- Utilice la tabla users para este ejercicio.

- La salida debe contener ``user_id``, ``user_dob``, ``user_email_id`` y ``user_day_of_birth``.

- Utilice funciones específicas de fecha para obtener el mes utilizando ``user_dob``.

- ``user_day_of_birth`` debe ser un día completo con el primer carácter en mayúsculas, por ejemplo `Tuesday`.

- Los datos deben ordenarse por día dentro del mes ``May``.

In [11]:
from tabulate import tabulate 
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
    SELECT user_id, user_dob, user_email_id,TO_CHAR(user_dob, 'FMDay') AS user_day_of_birth
    FROM users
    WHERE EXTRACT(MONTH FROM user_dob) = 5
    ORDER BY EXTRACT(DAY FROM user_dob);
    """);

r = cursor.fetchall()

print(tabulate(r, headers = ['user_id', 'user_dob', 'user_email_id', 'user_day_of_birth']))
connection.close()

  user_id  user_dob    user_email_id            user_day_of_birth
---------  ----------  -----------------------  -------------------
        4  1998-05-24  mtanswill3@dedecms.com   Sunday
       12  1983-05-26  uglayzerb@pinterest.com  Thursday
        1  1973-05-31  gbode0@imgur.com         Thursday
        2  2003-05-31  lgisbey1@mail.ru         Saturday


IX) Obtenga los nombres e ids de correo electrónico de los usuarios añadidos en el año 2019.

- Utilice la tabla users para este ejercicio.

- La salida debe contener ``user_id``, ``user_name``, ``user_email_id``, ``created_ts``, ``created_year``.

- Utilice funciones específicas de fecha para obtener el año utilizando ``created_ts``.

- ``user_name`` es una columna derivada de concatenar ``user_first_name`` y ``user_last_name`` con un espacio en medio.

- ``user_name`` debe tener valores en mayúsculas.

- Los datos deben ordenarse en forma ascendente por ``user_name``

In [12]:
from tabulate import tabulate 
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
    SELECT user_id, UPPER(CONCAT(user_first_name, ' ', user_last_name)) AS user_name, user_email_id,created_ts,EXTRACT(YEAR FROM created_ts) AS created_year
    FROM users
    WHERE EXTRACT(YEAR FROM created_ts) = 2019
    ORDER BY user_name ASC;
    """);

r = cursor.fetchall()

print(tabulate(r, headers = ['user_id', 'user_name', 'user_email_id', 'created_ts', 'created_year']))
connection.close()

  user_id  user_name        user_email_id              created_ts             created_year
---------  ---------------  -------------------------  -------------------  --------------
        8  CYBIL LISSIMORE  clissimore7@pinterest.com  2019-12-09 14:08:30            2019
       25  PHYLYS ASLIE     paslieo@qq.com             2019-10-01 01:34:28            2019
       12  UNA GLAYZER      uglayzerb@pinterest.com    2019-09-17 03:24:21            2019
       17  YANKEE JELF      yjelfg@wufoo.com           2019-09-16 16:09:12            2019


X) Obtenga el número de usuarios por género. Utilice la tabla de ``users`` para este ejercicio.

- La salida debe contener el ``gender`` y ``user_count``.

- Para los hombres la salida debe mostrar ``Male`` y para las mujeres la salida debe mostrar ``Female``.

- Si no se especifica el sexo, se mostrará ``Not Specified``.

- Los datos deben ordenarse en forma descendente por ``user_count``.

In [13]:
from tabulate import tabulate
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
   SELECT  CASE  WHEN user_gender = 'M' THEN 'Male' WHEN user_gender = 'F' THEN 'Female'ELSE 'Not Specified' END AS gender,COUNT(*) AS user_count
    FROM users
    GROUP BY gender
    ORDER BY user_count DESC;
    """);

r = cursor.fetchall()

print(tabulate(r, headers = ['user_gender', 'user_count']))
connection.close()

user_gender      user_count
-------------  ------------
Female                   13
Male                     10
Not Specified             2


XI) Obtenga los 4 últimos dígitos de los ids únicos.

- Utilice la tabla ``users`` para este ejercicio.

- El resultado debe contener ``user_id``, ``user_unique_id`` y ``user_unique_id_last4``.

- Los identificadores únicos son ``null`` o ``not null``.

- Los identificadores únicos contienen números y guiones y son de diferente longitud.

- Necesitamos obtener los últimos 4 dígitos descartando los guiones sólo cuando el número de dígitos es al menos 9.

- Si el identificador único es nulo, debe mostrarse ``Not Specified``.

- Después de descartar los guiones, si el identificador único tiene menos de 9 dígitos, debe mostrar ``Invalid Unique Id``.

- Los datos deben ordenarse por ``user_id``. Es posible que aparezca ``None`` o ``null`` para aquellos identificadores de usuario en los que no haya un identificador único para ``user_unique_id``.

In [14]:
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
   SELECT user_id, COALESCE(user_unique_id, 'Not Specified') AS user_unique_id, CASE 
        WHEN user_unique_id IS NULL THEN 'Not Specified'
        WHEN LENGTH(REPLACE(user_unique_id, '-', '')) >= 9 THEN RIGHT(REPLACE(user_unique_id, '-', ''), 4)
        ELSE 'Invalid Unique Id'
        END AS user_unique_id_last4
    FROM users
    ORDER BY user_id;
    """);

r = cursor.fetchall()

print(tabulate(r, headers = ['user_id', 'user_unique_id', 'user_unique_id_last4']))
connection.close()

  user_id  user_unique_id    user_unique_id_last4
---------  ----------------  ----------------------
        1  88833-8759        8759
        2  262501-029        1029
        3  391-33-2823       2823
        4  1195413-80        1380
        5  471-24-6869       6869
        6  192374-933        4933
        7  749-27-47-52      4752
        8  461-75-4198       4198
        9  892-36-676-2      6762
       10  197-54-1646       1646
       11  232-55-52-58      5258
       12  898-84-336-6      3366
       13  247-95-68-44      6844
       14  415-48-894-3      8943
       15  403-39-5-869      5869
       16  399-83-05-03      0503
       17  607-99-0411       0411
       18  430-01-578-5      5785
       19  571-09-6181       6181
       20  478-32-02-87      0287
       21  Not Specified     Not Specified
       22  Not Specified     Not Specified
       23  830-40-5287       5287
       24  778-0845          Invalid Unique Id
       25  368-44-4478       4478


XII) Obtenga el recuento de usuarios en función del código de país.

- Utilice la tabla ``users`` para este ejercicio.

- La salida debe contener el código de país y el recuento.

- No debe haber ningún ``+`` en el código de país. Sólo debe contener dígitos.

- Los datos deben ordenarse como números por código de país.

- Debemos descartar ``user_phone_no`` con valores ``null``.

In [15]:
from tabulate import tabulate 
connection = psycopg2.connect(user="Elianna_user",
                                  password="password",
                                  host="localhost",
                                  port="5432",
                                  database="Elianna_db")
cursor = connection.cursor()

cursor.execute("""
    SELECT CAST(LEFT(SUBSTRING(user_phone_no FROM '\+(\d+)'), POSITION('(' IN user_phone_no) - 1) AS INTEGER) AS country_code, COUNT(*) AS user_count
    FROM users
    WHERE user_phone_no IS NOT NULL
    GROUP BY country_code
    ORDER BY country_code;
    """);

r = cursor.fetchall()

print(tabulate(r, headers = ['country_code', 'user_count']))
connection.close()

  country_code    user_count
--------------  ------------
             1             1
             7             2
            48             1
            54             1
            55             1
            62             3
            63             1
            81             1
            84             1
            86             4
           229             1
           249             1
           351             1
           370             1
           380             1
           420             1
           598             1
